# Example : Using the magnetic probes

This example will show how to use the probes class.

## Generate a starting equilibrum (via a forward solve)

First, we need a tokamak and equilibrium since the probes take properties from the equilibrium as inputs. 

We will copy the code from example_1 to generate a sample equilibrium.

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

# build machine
from freegsnke import build_machine
tokamak = build_machine.tokamak(
    active_coils_path=f"../machine_configs/MAST-U/MAST-U_like_active_coils.pickle",
    passive_coils_path=f"../machine_configs/MAST-U/MAST-U_like_passive_coils.pickle",
    limiter_path=f"../machine_configs/MAST-U/MAST-U_like_limiter.pickle",
    wall_path=f"../machine_configs/MAST-U/MAST-U_like_wall.pickle",
    magnetic_probe_path=f"../machine_configs/MAST-U/MAST-U_like_magnetic_probes.pickle",
)

# initialise the equilibrium
from freegsnke import equilibrium_update
eq = equilibrium_update.Equilibrium(
    tokamak=tokamak,
    Rmin=0.1, Rmax=2.0,   # Radial range
    Zmin=-2.2, Zmax=2.2,  # Vertical range
    nx=65,                # Number of grid points in the radial direction (needs to be of the form (2**n + 1) with n being an integer)
    ny=129,               # Number of grid points in the vertical direction (needs to be of the form (2**n + 1) with n being an integer)
    # psi=plasma_psi
)  

# initialise the profiles
from freegsnke.jtor_update import ConstrainPaxisIp
profiles = ConstrainPaxisIp(
    eq=eq,        # equilibrium object
    paxis=8e3,    # profile object
    Ip=6e5,       # plasma current
    fvac=0.5,     # fvac = rB_{tor}
    alpha_m=1.8,  # profile function parameter
    alpha_n=1.2   # profile function parameter
)

# load the nonlinear solver
from freegsnke import GSstaticsolver
GSStaticSolver = GSstaticsolver.NKGSsolver(eq)    

# set the coil currents
import pickle
with open('simple_diverted_currents_PaxisIp.pk', 'rb') as f:
    current_values = pickle.load(f)
for key in current_values.keys():
    eq.tokamak[key].current = current_values[key]

# carry out the foward solve to find the equilibrium
GSStaticSolver.solve(eq=eq, 
                     profiles=profiles, 
                     constrain=None, 
                     target_relative_tolerance=1e-9)

# updates the plasma_psi (for later on)
eq._updatePlasmaPsi(eq.plasma_psi)

# plot the resulting equilbria 
fig1, ax1 = plt.subplots(1, 1, figsize=(4, 8), dpi=80)
ax1.grid(True, which='both')
eq.plot(axis=ax1, show=False)
eq.tokamak.plot(axis=ax1, show=False)
ax1.set_xlim(0.1, 2.15)
ax1.set_ylim(-2.25, 2.25)
plt.tight_layout()

## Using the probe objects

The tokamak object has a probe object attribute pre-initialised, however, if we wanted to we could create a stand alone one by importing `freegsnke.magnetic_probes` and then setting up with `probes = magnetic_probes.Probe()`. 

In the tokamak, the probes object is located in `tokamak.probes`. When initialised it reads the information on the names, positions and orientations of the different probes from the pickle file. 

We need to initialise the Greens functions that are used in the calculations. This is done by running `tokamak.probes.initialise_setup(eq)`. This takes an input equilibrium object and saves probe attributes for each current source (i.e. the coils and the plasma) and evaluates the Greens functions at the positions of the probes. The purpose of the equilibrium here is to provide the size/shape of the grid that is used when determining the plasma Greens functions. 

In [ ]:
tokamak.probes.initialise_setup(eq)

Once initialised we can also now access information from the different probes:
- **flux loops**  measure $\psi(r,z)$.
- **pickup coils** measure $\vec B(r,z)\cdot \hat n$ where $\hat n$ is the orientation vector of the pickup coil.

In [ ]:
# display the first five flux loop info
tokamak.probes.floops[0:5]
#print(tokamak.probes.floop_order[:5])
#print(tokamak.probes.floop_pos[:5])

In [ ]:
# display the first five pickup coils info
tokamak.probes.pickups[0:5]
#print(tokamak.probes.floop_order[:5])
#print(tokamak.probes.floop_pos[:5])

Now in principle we could update and re-solve for the equilibrium. Doing this will not require any changes to the probe object, assuming the machine setup doesn't change. 

Once we have the equilibrium we want to analyse with the probes, we can call the probe functions `calculate_fluxloop_value(eq)` and `calculate_pickup_value(eq)`, the outputs of which are arrays with probe values for each probe. 

If one is interested in certain probes, then `tokamak.probes.floop_order` and `tokamak.probes.pickup_order` contain a list of the probe names which can be used to find the appropriate element of the output list. Alternatively the could be combined into a dictionary.

In [ ]:
# compute probe values from equilibrium 
floops_vals = tokamak.probes.calculate_fluxloop_value(eq)
pickups_vals = tokamak.probes.calculate_pickup_value(eq)

In [ ]:
# create dictionary to access specific values (show here for the flux loops)
dict = {}
for i, el in enumerate(tokamak.probes.floop_order):
    dict[el] = floops_vals[i]
dict

Suppose we want to compute a new equilibrium with a different grid spacing or shape. We don't need to update the probe objects, we simply pass the new equilibrium to the 'calculate' functions. The first time a new grid is encountered it will create a new set of greens functions and save them to a dictionary so that they can be reused in the future if the same grid is used again. 

Below is a new equilibrium with a modified grid shape and spacing. When a new grid is encountered, a message is displayed to tell that new greens are computed. Note it only does it the first time.

Note that computing on a different grid but with same plasma setup should give the same values at the probes (which it does). 

In [ ]:
# initialise the equilibrium
eq_new = equilibrium_update.Equilibrium(
    tokamak=tokamak,
    Rmin=0.1, Rmax=2.0,   # Radial range
    Zmin=-2.0, Zmax=2.0,  # Vertical range
    nx=65,                # Number of grid points in the radial direction (needs to be of the form (2**n + 1) with n being an integer)
    ny=129,               # Number of grid points in the vertical direction (needs to be of the form (2**n + 1) with n being an integer)
    # psi=plasma_psi
)  


from freegsnke.jtor_update import ConstrainPaxisIp
profiles = ConstrainPaxisIp(
    eq=eq_new,
    paxis=8e3,
    Ip=6e5,
    fvac=0.5,
    alpha_m=1.8,
    alpha_n=1.2
)

from freegsnke import GSstaticsolver
GSStaticSolver = GSstaticsolver.NKGSsolver(eq_new)    

import pickle
with open('simple_diverted_currents_PaxisIp.pk', 'rb') as f:
    current_values = pickle.load(f)
for key in current_values.keys():
    eq_new.tokamak[key].current = current_values[key]

GSStaticSolver.solve(eq=eq_new, 
                     profiles=profiles, 
                     constrain=None, 
                     target_relative_tolerance=1e-9)

In [ ]:
floops_vals_new = tokamak.probes.calculate_fluxloop_value(eq_new)
pickups_vals_new = tokamak.probes.calculate_pickup_value(eq_new)

In [ ]:
# compare values
print(floops_vals[:5])
print(floops_vals_new[:5])

In [ ]:
# compare values
print(pickups_vals[:5])
print(pickups_vals_new[:5])

If we re-run this same line of code, we don't get the message that the greens functions have been recalculated. They are stored in a dictionary labeled by a key containing the grid specification in the form `key = (Rmin,Rmax,Zmin,Zmax,nx,ny)`.

In [ ]:
pickup_vals_new2 = tokamak.probes.calculate_pickup_value(eq_new)

# show greens function keys
tokamak.probes.greens_B_plasma_oriented.keys()


We can also plot the fluxloop and the pickup coil locations (and orientations) on our machine model. 

In [ ]:
# plot the resulting equilbria 
fig1, ax1 = plt.subplots(1, 1, figsize=(4, 8), dpi=80)
ax1.grid(True, which='both')
# eq.plot(axis=ax1, show=False)
eq.tokamak.plot(axis=ax1, show=False)
eq.tokamak.probes.plot(axis=ax1, show=False, floops=True, pickups=True, pickups_scale=0.05)
ax1.plot(tokamak.limiter.R, tokamak.limiter.Z, color='k', linewidth=1.2, linestyle="--")
ax1.plot(tokamak.wall.R, tokamak.wall.Z, color='k', linewidth=1.2, linestyle="-")
ax1.set_xlim(0.1, 2.15)
ax1.set_ylim(-2.25, 2.25)
plt.tight_layout()